In [1]:
import pandas as pd
import numpy as np
import xlsxwriter as xl
import re
import itertools
from datetime import datetime
pd.options.mode.chained_assignment = None



## TODO create a front end interface to select which tool and contest date
# tool = "DK"
tool = "Yahoo"
# sport = "NHL"
sport = "NBA"
contest_date = "dec4"

path = f"csv/{tool}_{sport}/{contest_date}.csv"


def print_current_df(dataframe):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        display(dataframe)

In [2]:
original_df = pd.read_csv(path)
if tool == "DK":
    original_df[['Game','Date','Time','Zone']] = original_df.loc[:,'Game Info'].str.split(" ", 3, expand = True)
    original_df = original_df.rename(columns={'AvgPointsPerGame':'FPPG','TeamAbbrev':'Team'})
print_current_df(original_df)

,ID,First Name,Last Name,Position,Team,Opponent,Game,Time,Salary,FPPG,Injury Status,Starting
0,nba.p.4614,DeMar,DeRozan,PF,CHI,BKN,CHI@BKN,8:00PM EST,38,40.7,,
1,nba.p.4897,Nikola,Vucevic,C,CHI,BKN,CHI@BKN,8:00PM EST,34,39.1,,
2,nba.p.5324,Zach,LaVine,SG,CHI,BKN,CHI@BKN,8:00PM EST,33,37.4,,
3,nba.p.5747,Derrick,Jones Jr.,SF,CHI,BKN,CHI@BKN,8:00PM EST,10,15.0,,
4,nba.p.5754,Alex,Caruso,SG,CHI,BKN,CHI@BKN,8:00PM EST,19,26.1,GTD,
5,nba.p.5764,Lonzo,Ball,PG,CHI,BKN,CHI@BKN,8:00PM EST,25,31.1,,
6,nba.p.5841,Tony,Bradley,C,CHI,BKN,CHI@BKN,8:00PM EST,10,10.4,,
7,nba.p.6026,Troy,Brown Jr.,SF,CHI,BKN,CHI@BKN,8:00PM EST,10,6.5,,
8,nba.p.6061,Alize,Johnson,PF,CHI,BKN,CHI@BKN,8:00PM EST,10,6.0,,
9,nba.p.6169,Coby,White,PG,CHI,BKN,CHI@BKN,8:00PM EST,11,11.3,INJ,


### Modify Original CSV entries to create a DF of relevant data


In [3]:
def convert_positions_to_values(df,original_df):
    
    df.insert(loc=1, column='Pos', value=0)
    conditions = [
        (df['Position'] == 'PG'),
        (df['Position'] == 'SG'),
        (df['Position'] == 'SF'),
        (df['Position'] == 'PF'),
        (df['Position'] == 'C')
    ]
    values = [1, 2, 3, 4, 5]
    df.loc[:,'Pos'] = np.select(conditions, values)

    
def single_game(df,original_df):
    df = original_df[['FPPG','Salary']]
    df.insert(0, 'Name',original_df["First Name"] + ' ' + original_df["Last Name"])
    
def drop_inactive_players(df):
    df = df[df['Injury Status'] != 'INJ'] 
    df = df[df['Injury Status'] != 'O'] 
    df = df[df['Injury Status'] != 'IR'] 
    return df
def create_df(sport,tool,original_df):
    if tool == "Yahoo":
        df = original_df[['Position', 'Team' ,'Opponent','Injury Status','FPPG','Salary']].copy()
        df.insert(3, 'Name',original_df["First Name"] + ' ' + original_df["Last Name"])
        df = drop_inactive_players(df)
        if sport == "NHL":
            df.insert(len(df.columns),'Starting',original_df['Starting'])
            df['Value'] = df['FPPG']/df['Salary']
        else:
            number_of_teams = len(original_df['Team'].value_counts())
            if(number_of_teams == 2):
                single_game(df,original_df)
            else:    
                convert_positions_to_values(df,original_df)
                df.head()
    if tool == "DK":
        df = original_df[['Position', 'Name','Team','Game','Time','FPPG','Salary']]
    print_current_df(df)
    return df
    
df = create_df(sport,tool,original_df)

,Position,Pos,Team,Opponent,Name,Injury Status,FPPG,Salary
0,PF,4,CHI,BKN,DeMar DeRozan,,40.7,38
1,C,5,CHI,BKN,Nikola Vucevic,,39.1,34
2,SG,2,CHI,BKN,Zach LaVine,,37.4,33
3,SF,3,CHI,BKN,Derrick Jones Jr.,,15.0,10
4,SG,2,CHI,BKN,Alex Caruso,GTD,26.1,19
5,PG,1,CHI,BKN,Lonzo Ball,,31.1,25
6,C,5,CHI,BKN,Tony Bradley,,10.4,10
7,SF,3,CHI,BKN,Troy Brown Jr.,,6.5,10
8,PF,4,CHI,BKN,Alize Johnson,,6.0,10
10,SG,2,CHI,BKN,Matt Thomas,,2.5,10


### Interface

To be modified as more front end functionality is added

In [4]:
def add_to_lineup(df, player_df, name):
    if player_df['Name'].str.lower().str.contains(name.lower()).any() != True:
        player_df = player_df.append(df.loc[df['Name'].str.lower() == name.lower()])
    else:
        if name != '' : print(name + " is already in list")
    return player_df
def remove_from_lineup(player_df, name):
    print("name? " + name)
    if player_df['Name'].str.lower().str.contains(name.lower()).any() == True:
        player_df.drop(player_df.loc[player_df['Name'].str.lower()==name.lower()].index, inplace=True)
    else:
        if name != ''  : print(name + " is not in in list") 
    return player_df

In [5]:
df2 = df.sort_values(by=['Salary'], ascending = False)
if sport == 'NHL':
    df2 = df2.drop(df2[df2.Starting == "No"].index)
for name in df2['Name']: 
    print(name, end = ", ")

Kevin Durant, James Harden, Stephen Curry, Paul George, Dejounte Murray, Jayson Tatum, DeMar DeRozan, CJ McCollum, Nikola Vucevic, Zach LaVine, De'Aaron Fox, Tyler Herro, Khris Middleton, Jaren Jackson Jr., Kyle Lowry, Jrue Holiday, Draymond Green, Lonzo Ball, Jusuf Nurkic, Dillon Brooks, Bobby Portis, Jordan Poole, Andrew Wiggins, Jakob Poeltl, Tyrese Haliburton, Derrick White, Marcus Smart, Keldon Johnson, Richaun Holmes, Robert Williams, Dennis Schroder, Jalen Brunson, Reggie Jackson, Tim Hardaway Jr., LaMarcus Aldridge, Buddy Hield, Alex Caruso, Desmond Bane, Norman Powell, Ivica Zubac, De'Anthony Melton, Eric Bledsoe, Grayson Allen, Pat Connaughton, Steven Adams, Tyus Jones, Chimezie Metu, Marcus Morris Sr., Dorian Finney-Smith, Dewayne Dedmon, DeAndre' Bembry, Patty Mills, Davion Mitchell, Maxi Kleber, Terence Davis, Isaiah Hartenstein, Santi Aldama, Bruce Brown, Terance Mann, Robert Covington, Dennis Smith Jr., Damian Jones, Duncan Robinson, Greg Brown, P.J. Tucker, Marvin Bagle

### Table Add

In [6]:
def get_names(names):
    return [x.strip() for x in names.split(',')]


##adding these blank columns that can be edited in an excel sheet to add the values
##Future versions will let user
def add_extra_columns(player_df):
    player_df['Worst'] = ' '
    player_df['Best'] = ' '
    if sport == 'NBA':
        player_df['Probable'] = ' '
    player_df['Actual'] = ' '
    if sport == 'NBA':
        player_df['Deviation'] = ' '
        player_df['Value'] = 1
        player_df['Actual Value'] = ' '
    return player_df
def choose_relevant_players(player_df,df):
    queries = ['exit', 'all', 'show']
    player_df = player_df.replace(np.nan, '', regex=True)
    
    exit = False
    while exit!=True:
        query = input('Name Of Player:')
        if query not in queries:
            if re.search('^remove', query):
                names_to_remove = re.findall('(?<=remove).*$', query)[0]
                names = get_names(names_to_remove)
                for name in names:
                    name = name.title()
                    player_df = remove_from_lineup(player_df,name)
                    print(f"removed {name}")

            else:
                names = get_names(query)
                for name in names:
                    name = name.title()
                    if df['Name'].str.lower().str.contains(name.lower()).any():
                        player_df = add_to_lineup(df,player_df,name)   
                    else:
                        print(name + " is not in the csv list. Please try again")
        elif query =='show':
            print_current_df(player_df)
        elif query =='all':
            player_df = df
            print_current_df(player_df)
            exit = True
        else:
            exit = True
    return player_df
player_df = pd.DataFrame(columns = df.columns)
player_df = add_extra_columns(player_df)
player_df = choose_relevant_players(player_df,df)


Name Of Player:Stephen Curry
Name Of Player:James Harden
Name Of Player:show


,Position,Pos,Team,Opponent,Name,Injury Status,FPPG,Salary,Worst,Best,Probable,Actual,Deviation,Value,Actual Value
86,PG,1,GS,SA,Stephen Curry,,47.3,48,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,PG,1,BKN,CHI,James Harden,,46.0,49,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Name Of Player:Stephen Curry
Stephen Curry is already in list
Name Of Player:Remove James Harden
Remove James Harden is not in the csv list. Please try again
Name Of Player:Add Stephan Adams
Add Stephan Adams is not in the csv list. Please try again
Name Of Player:Stephen Adams
Stephen Adams is not in the csv list. Please try again
Name Of Player:Kevin Durant
Name Of Player:show


,Position,Pos,Team,Opponent,Name,Injury Status,FPPG,Salary,Worst,Best,Probable,Actual,Deviation,Value,Actual Value
86,PG,1,GS,SA,Stephen Curry,,47.3,48,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,PG,1,BKN,CHI,James Harden,,46.0,49,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,PF,4,BKN,CHI,Kevin Durant,,46.8,51,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Name Of Player:remove James Harden
name? James Harden
removed James Harden
Name Of Player:show


...

Name Of Player:remove Kevin Durant
name? Kevin Durant
removed Kevin Durant
Name Of Player:show


,Position,Pos,Team,Opponent,Name,Injury Status,FPPG,Salary,Worst,Best,Probable,Actual,Deviation,Value,Actual Value
86,PG,1,GS,SA,Stephen Curry,,47.3,48,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,PG,1,BKN,CHI,James Harden,,46.0,49,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,PG,1,SA,GS,Dejounte Murray,,46.5,44,NaN,NaN,NaN,NaN,NaN,NaN,NaN
137,PG,1,MIA,MIL,Kyle Lowry,,31.2,28,NaN,NaN,NaN,NaN,NaN,NaN,NaN
154,PG,1,MIL,MIA,Jrue Holiday,,32.8,28,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Name Of Player: Lonzo Ball, Marcus Smart, Dennis Schroder, Jalen Brunson, Tyus Jones, Gary Payton II, CJ McCollum, Tyler Herro, Derrick White, Tyrese Haliburton, Desmond Bane, De'Anthony Melton, Josh Richardson, Terence Davis, Jayson Tatum, Paul George, Khris Middleton, Dillon Brooks, Terance Mann, Norman Powell, Kevin Durant, Maxi Kleber, Chimezie Metu, Marcus Morris Sr., Robert Covington, Juan Toscano-Anderson, Jakob Poeltl, Jusuf Nurkic, Bobby Portis, LaMarcus Aldridge, Ivica Zubac, Dewayne Dedmon
Name Of Player:show


,Position,Pos,Team,Opponent,Name,Injury Status,FPPG,Salary,Worst,Best,Probable,Actual,Deviation,Value,Actual Value
86,PG,1,GS,SA,Stephen Curry,,47.3,48,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,PG,1,BKN,CHI,James Harden,,46.0,49,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,PG,1,SA,GS,Dejounte Murray,,46.5,44,NaN,NaN,NaN,NaN,NaN,NaN,NaN
137,PG,1,MIA,MIL,Kyle Lowry,,31.2,28,NaN,NaN,NaN,NaN,NaN,NaN,NaN
154,PG,1,MIL,MIA,Jrue Holiday,,32.8,28,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,PG,1,CHI,BKN,Lonzo Ball,,31.1,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN
173,PG,1,BOS,POR,Marcus Smart,,29.9,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN
171,PG,1,BOS,POR,Dennis Schroder,,29.0,21,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63,PG,1,DAL,MEM,Jalen Brunson,,27.7,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,PG,1,MEM,DAL,Tyus Jones,,16.5,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Name Of Player:exit


In [9]:
def output_sheet(output_df):
#     dt = datetime.now()
#     now = dt.strftime("%Y-%m-%d %H_%M")

    file_name = f'output/{tool}/{sport}/{contest_date}.xlsx'
    # Create a Pandas Excel writer using XlsxWriter as the engine.
    writer = pd.ExcelWriter(file_name, engine='xlsxwriter')

    # Convert the dataframe to an XlsxWriter Excel object. Turn off the default
    # header and index and skip one row to allow us to insert a user defined
    # header.

    output_df.to_excel(writer, index = False, header=True)

    # Get the xlsxwriter workbook and worksheet objects.
    workbook = writer.book
    worksheet = writer.sheets['Sheet1']

    # Get the dimensions of the dataframe.
    (max_row, max_col) = output_df.shape

    # Create a list of column headers, to use in add_table().
    column_settings = []
    for header in player_df.columns:
        column_settings.append({'header': header})

    # Add the table.
    worksheet.add_table(0, 0, max_row, max_col - 1, {'columns': column_settings})

    # Make the columns wider for clarity.
    worksheet.set_column(0, max_col - 1, 12)

    if sport == 'NHL': 
        superstar_df = df.loc[df['Salary'] > 18]
        superstar_df.to_excel (writer, index = False, header=True, sheet_name = "Superstars")
        worksheet2 = writer.sheets['Superstars']

        (max_row, max_col) = df.shape
        print_current_df(superstar_df)
        worksheet2.add_table(0, 0, max_row, max_col - 1, {'columns': column_settings})

        # Make the columns wider for clarity.
        worksheet2.set_column(0, max_col - 1, 12)

    # Close the Pandas Excel writer and output the Excel file.
    writer.close()

output_sheet(player_df)

### Single-game analysis [WIP]

#### Single Game Entry Strategy:

- **Stars:** Out of the 4-5 likely star players, choose 3.

- **Peasants:** Out of the 5-10 non-star players, choose 5.



In [8]:
single_df = filled_df[['Name','Salary','Worst','Best','Probable']]
print_current_df(single_df)
create_lineup(single_df)

NameError: name 'filled_df' is not defined

In [ ]:
max_salary = 30

In [ ]:
## best 5 probable, best 5 worst, best 5 best. Create combinations of lineups with the most recurring names

def create_lineup(df):
    lineup = []
    df2 = df.ge()
        
    
    print(highest_paid_player.Name)
    

In [ ]:
numbers = [1, 2, 3, 7, 7, 9, 10]
result = [seq for i in range(len(numbers), 0, -1) for seq in itertools.combinations(numbers, i) if sum(seq) >= 10]
print(result)

In [ ]:
class Lineup: 
    superstar = ""
    megastar = ""
    star = ""
    other_players = []   
    

In [ ]:
class MyClass:
    """A simple example class"""
    i = 12345

    def __init__(self,data):
        self.data = data
l = [1,2,"e"]
x = MyClass(l)
x.data


### Multi-game analysis [WIP]

In [ ]:
# sorted_df = filled_df.sort_values(by=['Pos','Probable'],ascending=[True,False]).drop(['Notes', 'Pts', 'Reb', 'Ast', 'Stl', 'Blk', 'TO',
#        'Total'],axis = 1)

# # sorted_df.insert(12,'Worst Value',sorted_df["Worst"] / sorted_df["Salary"])
# # sorted_df.insert(13,'Best Value',sorted_df["Best"] / sorted_df["Salary"])
# df=sorted_df
# lineups = []
# positions = []



# df = df.loc[:, lambda df: ['Pos','Salary','Name', "Worst","Best",'Probable']]
# [positions.append(df.loc[(sorted_df['Pos'] == i)].sort_values(by=['Pos','Probable'],ascending=[True,False])) for i in range(1,6)]

# [print_current_df(position_df) for position_df in positions]

In [ ]:
def select_eligible(df):
    = sorted_df.loc[(sorted_df['Salary'] > 30)]
df

In [ ]:
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))


In [ ]:
# d = pd.concat(
#     [df.loc[a].reset_index(), df.loc[b].reset_index()],
#     keys=['a', 'b'], axis=1
# )

# d
